In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
# Importer la bibliothèque pandas

movies_df = pd.read_csv('clean_data.csv')

print("Colonnes dispo :", movies_df.columns)

print(movies_df.head())

Colonnes dispo : Index(['id', 'year', 'title', 'runtime', 'collection', 'genres', 'tagline',
       'overview', 'cast', 'director', 'producer', 'keywords', 'adult',
       'production_companies', 'spoken_languages', 'popularity', 'vote_count',
       'vote_average'],
      dtype='object')
      id  year                        title  runtime  \
0    862  1995                    Toy Story       81   
1   8844  1995                      Jumanji      104   
2  15602  1995             Grumpier Old Men      101   
3  31357  1995            Waiting to Exhale      127   
4  11862  1995  Father of the Bride Part II      106   

                                      collection  \
0                           Toy Story Collection   
1                             Jumanji Collection   
2                      Grumpy Old Men Collection   
3                                            NaN   
4  Father of the Bride (Steve Martin) Collection   

                                           genres  \
0  ['An

In [3]:
# Importer la bibliothèque pandas

movies_df = pd.read_csv('clean_data.csv')

print("Colonnes dispo :", movies_df.columns)

print(movies_df.head())

Colonnes dispo : Index(['id', 'year', 'title', 'runtime', 'collection', 'genres', 'tagline',
       'overview', 'cast', 'director', 'producer', 'keywords', 'adult',
       'production_companies', 'spoken_languages', 'popularity', 'vote_count',
       'vote_average'],
      dtype='object')
      id  year                        title  runtime  \
0    862  1995                    Toy Story       81   
1   8844  1995                      Jumanji      104   
2  15602  1995             Grumpier Old Men      101   
3  31357  1995            Waiting to Exhale      127   
4  11862  1995  Father of the Bride Part II      106   

                                      collection  \
0                           Toy Story Collection   
1                             Jumanji Collection   
2                      Grumpy Old Men Collection   
3                                            NaN   
4  Father of the Bride (Steve Martin) Collection   

                                           genres  \
0  ['An

In [4]:
# Nettoyer la colonne 'genres' en supprimant les crochets et les guillemets
movies_df['genres'] = movies_df['genres'].str.strip("[]").str.replace("'", "").str.replace(",", " ")

# Supprimer les lignes où 'genres' est vide ou ne contient aucune information pertinente
movies_df = movies_df[movies_df['genres'].str.strip() != ""]

# Nettoyer la colonne 'keywords' en supprimant les crochets et les guillemets
movies_df['keywords'] = movies_df['keywords'].str.strip("[]").str.replace("'", "").str.replace(",", " ")

# Nettoyer la colonne 'collection' en supprimant les crochets et les guillemets, et remplacer NaN par une chaîne vide
movies_df['collection'] = movies_df['collection'].fillna("").str.strip("[]").str.replace("'", "").str.replace(",", " ")

movies_df['overview'] = movies_df['overview'].fillna("").str.strip()

# Vérifier les valeurs uniques dans la colonne 'adult' pour confirmer son contenu
print("Valeurs uniques dans la colonne 'adult':", movies_df['adult'].unique())

# Vérifier le nombre de films après filtrage
print(f"Nombre de films après filtrage des genres vides : {len(movies_df)}")


Valeurs uniques dans la colonne 'adult': [False]
Nombre de films après filtrage des genres vides : 100


In [5]:

# Combiner 'genres', 'keywords', et 'overview' pour créer la colonne 'features'
movies_df['features'] = (
    movies_df['genres'] + " " + 
    movies_df['keywords'] + " " +
    movies_df['overview']
)

# Vérifier les premières lignes des features combinées
print("Colonnes combinées (avec 'overview') :")
print(movies_df[['title', 'features']].head())

Colonnes combinées (avec 'overview') :
                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            features  
0  Animation  Adventure  Family  Comedy martial a...  
1  Adventure  Fantasy  Family giant insect  board...  
2  Romance  Comedy fishing  halloween  sequel  ol...  
3  Comedy  Drama  Romance based on novel or book ...  
4  Comedy  Family parent child relationship  baby...  


In [6]:
# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['features'])

# Calcul de la similarité cosine
similarity_matrix = cosine_similarity(tfidf_matrix)

# Créer un DataFrame pour représenter la similarité
similarity_df = pd.DataFrame(similarity_matrix, index=movies_df['title'], columns=movies_df['title'])

# Vérifier les premières lignes de la matrice
print("Matrice de similarité :")
print(similarity_df.head())


Matrice de similarité :
title                        Toy Story   Jumanji  Grumpier Old Men  \
title                                                                
Toy Story                     1.000000  0.028174          0.025018   
Jumanji                       0.028174  1.000000          0.033798   
Grumpier Old Men              0.025018  0.033798          1.000000   
Waiting to Exhale             0.012977  0.016763          0.031110   
Father of the Bride Part II   0.007404  0.005013          0.055279   

title                        Waiting to Exhale  Father of the Bride Part II  \
title                                                                         
Toy Story                             0.012977                     0.007404   
Jumanji                               0.016763                     0.005013   
Grumpier Old Men                      0.031110                     0.055279   
Waiting to Exhale                     1.000000                     0.013921   
Father of t

In [7]:
def recommend_and_explain_with_threshold(movie_title, movies_df, similarity_df, top_n=5, threshold=0.15):
    """
    Recommande les films les plus similaires à un film donné avec un seuil de score.

    :param movie_title: Titre du film de référence
    :param movies_df: DataFrame contenant les informations des films
    :param similarity_df: Matrice de similarité (DataFrame)
    :param top_n: Nombre de recommandations à retourner
    :param threshold: Score de similarité minimal pour considérer une recommandation
    :return: None
    """
    if movie_title not in similarity_df.index:
        print(f"Le film '{movie_title}' n'existe pas dans la base de données.")
        return None

    similar_movies = similarity_df[movie_title].sort_values(ascending=False).drop(movie_title)

    print(f"Top {top_n} films similaires à '{movie_title}':\n")
    count = 0
    for recommended_title, similarity_score in similar_movies.items():
        if similarity_score < threshold:
            break  # Arrêter si le score est en dessous du seuil

        if count >= top_n:
            break

        original_genres = movies_df.loc[movies_df['title'] == movie_title, 'genres'].values[0]
        recommended_genres = movies_df.loc[movies_df['title'] == recommended_title, 'genres'].values[0]
        common_genres = set(original_genres.split()) & set(recommended_genres.split())

        original_keywords = movies_df.loc[movies_df['title'] == movie_title, 'keywords'].values[0]
        recommended_keywords = movies_df.loc[movies_df['title'] == recommended_title, 'keywords'].values[0]

        original_collection = movies_df.loc[movies_df['title'] == movie_title, 'collection'].values[0]
        recommended_collection = movies_df.loc[movies_df['title'] == recommended_title, 'collection'].values[0]

        print(f"{count + 1}. {recommended_title} (Score de similarité : {similarity_score:.2f})")
        print(f"   -> Genres communs : {common_genres}")
        print(f"   -> Mots-clés communs : {set(original_keywords.split()) & set(recommended_keywords.split())}")
        print(f"   -> Collection commune : {'Oui' if original_collection == recommended_collection and original_collection else 'Non'}\n")

        count += 1

# Exemple : afficher les recommandations pour "Toy Story" avec un seuil
recommend_and_explain_with_threshold("Toy Story", movies_df, similarity_df, threshold=0.2)


Top 5 films similaires à 'Toy Story':

1. The Indian in the Cupboard (Score de similarité : 0.20)
   -> Genres communs : {'Family', 'Adventure'}
   -> Mots-clés communs : {'toy', 'life', 'comes', 'to'}
   -> Collection commune : Non



In [8]:
def search_similarity_by_keywords_with_threshold(keywords, movies_df, tfidf_vectorizer, tfidf_matrix, top_n=5, threshold=0.1):
    """
    Rechercher les films similaires à partir de mots-clés avec un seuil de similarité.

    :param keywords: Chaîne de mots-clés pour la recherche.
    :param movies_df: DataFrame contenant les informations des films.
    :param tfidf_vectorizer: Instance de TfidfVectorizer utilisée pour la vectorisation.
    :param tfidf_matrix: Matrice TF-IDF calculée sur la colonne 'features'.
    :param top_n: Nombre de recommandations à retourner.
    :param threshold: Score de similarité minimal pour considérer une recommandation.
    :return: None
    """
    # Vectoriser les mots-clés donnés
    keywords_vector = tfidf_vectorizer.transform([keywords])

    # Calculer la similarité cosine entre les mots-clés et tous les films
    similarity_scores = cosine_similarity(keywords_vector, tfidf_matrix)

    # Trier les films par similarité
    similar_movies_indices = similarity_scores[0].argsort()[::-1]  # Tri décroissant
    similar_movies = movies_df.iloc[similar_movies_indices]

    # Filtrer les films avec un seuil de similarité
    filtered_movies = [(index, row, similarity_scores[0][index]) 
                       for index, row in similar_movies.iterrows() 
                       if similarity_scores[0][index] >= threshold]

    # Afficher les résultats
    print(f"Top {top_n} films similaires à partir des mots-clés '{keywords}' (Seuil : {threshold}):\n")
    count = 0
    for index, row, similarity_score in filtered_movies:
        if count >= top_n:
            break

        print(f"{count + 1}. {row['title']} (Score de similarité : {similarity_score:.2f})")
        print(f"   -> Genres : {row['genres']}")
        print(f"   -> Mots-clés : {row['keywords']}")
        print(f"   -> Collection : {row['collection'] if row['collection'] else 'Aucune'}\n")
        count += 1

    # Vérification si aucune recommandation pertinente
    if count == 0:
        print("Aucune recommandation pertinente trouvée au-dessus du seuil spécifié.")

# Exemple : recherche basée sur les mots-clés avec seuil
search_similarity_by_keywords_with_threshold("foot", movies_df, tfidf_vectorizer, tfidf_matrix, threshold=0.1)


Top 5 films similaires à partir des mots-clés 'foot' (Seuil : 0.1):

1. Fair Game (Score de similarité : 0.12)
   -> Genres : Action
   -> Mots-clés : bomb  miami  florida  spy  detective  handcuffs  based on novel or book  hostage  fbi  kidnapping  hacker  kgb  chase  sadism  parking garage  pizza  police protection  psychopath  remake  car crash  conspiracy  on the run  fugitive  shootout  gunfight  sadist  foot chase  police detective  police station  held at gunpoint  double cross  cult film  machine gun  grenade launcher  rogue agent  woman lawyer  damsel in distress
   -> Collection : Aucune

2. From Dusk Till Dawn (Score de similarité : 0.12)
   -> Genres : Horror  Action  Thriller  Crime
   -> Mots-clés : dancing  sibling relationship  showdown  sheriff  bank robber  vampire  holy water  siege  stripper  priest  bank robbery  preacher  hostage situation  crucifix  recreational vehicle  mexican american border  female stripping  boa constrictor  foot fetish  barefoot
   -> Colle